# 'with' statement
- very common pattern in software is to aquire some kind of resource or context,
use it for awhile, then return it or undo it. 
- common examples are file and network descriptors
    - very important to use descriptors correctly
    - running out of descriptors can crash a server

# Context Management Protocol 
- ```__enter__``` - acquire resource
- ```__exit__``` - release resource

In [ ]:
# tedious to write all this out
import tempfile

tmp = tempfile.NamedTemporaryFile().name

fd = open(tmp, 'w')
# do things that might fail somehow
try:
    fd.write('foo')
finally: 
    # error or not, want to close the file descriptor
    # finally clause guarantees close will happen
    fd.close()

In [ ]:
# instead, use 'with'
# less work, more consise

with open(tmp, 'w') as fd:
    fd.write('foo')


# 'with' implements 'context manager' protocol
- like iteration protocol, a general protocol implemented by many classes
- ```__enter__``` method - called at start of with block
- ```__exit__``` method - called at end of with block, or when error raised

In [ ]:
class File():

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print('enter')
        self.open_file = open(self.filename, self.mode)
        # as variable bound to this
        return self.open_file

    def __exit__(self, *args):
        print('exit')
        self.open_file.close()

import tempfile
        
with File('/tmp/foo.txt', 'w') as fd:
    print('here')
    fd.write('foo')
    print('there')


In [ ]:
# 'with' use above roughly equivalent to:

f = File(tmp, 'w')
fd = f.__enter__()
try:
    fd.write('foo')
finally:
    # always executed, closes the file descriptor
    f.__exit__()


# Example - dominate module
- surpising and elegant implementation of a DOM builder using 'with' context protocol
- [doc](https://github.com/Knio/dominate)

In [ ]:
import dominate
from dominate.tags import *

doc = dominate.document(title='Dominate your HTML')

with doc.head:
    link(rel='stylesheet', href='style.css')
    script(type='text/javascript', src='script.js')

with doc:
    with div(id='header').add(ol()):
        for i in ['home', 'about', 'contact']:
            li(a(i.title(), href='/%s.html' % i))

    with div():
        attr(cls='body')
        p('Lorem ipsum..')

print(doc)

In [ ]:
# dominate also uses decorators...

@div
def greeting(name):
    p('Hello %s' % name)
print(greeting('Bob'))